In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [2]:
# Carregar o dataset Iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Dividir em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter os dados para tensores do PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)


In [3]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 10)  # 4 características de entrada
        self.fc2 = nn.Linear(10, 3)   # 3 classes de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 100  # Número de épocas para treinamento

for epoch in range(num_epochs):
    model.train()  # Colocar o modelo em modo de treinamento
    optimizer.zero_grad()  # Zerando os gradientes

    outputs = model(X_train_tensor.to(device))  # Passar os dados para o modelo
    loss = criterion(outputs, y_train_tensor.to(device))  # Calcular a perda
    loss.backward()  # Backpropagation
    optimizer.step()  # Atualizar os pesos

    if (epoch + 1) % 10 == 0:  # Exibir a perda a cada 10 épocas
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.7989
Epoch [20/100], Loss: 0.5326
Epoch [30/100], Loss: 0.3679
Epoch [40/100], Loss: 0.2889
Epoch [50/100], Loss: 0.2319
Epoch [60/100], Loss: 0.1826
Epoch [70/100], Loss: 0.1449
Epoch [80/100], Loss: 0.1175
Epoch [90/100], Loss: 0.0986
Epoch [100/100], Loss: 0.0858


In [5]:
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    correct = 0
    total = 0
    outputs = model(X_test_tensor.to(device))
    _, predicted = torch.max(outputs.data, 1)
    total += y_test_tensor.size(0)
    correct += (predicted == y_test_tensor.to(device)).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 100.00%
